# InceptionV3

Model to classify the food 101 images, based on the Inception V3
<br>
Tested and working with tensorflow 2.1.0 & keras 2.3.1 
- 67988 training images
- 22716 test images


In [9]:
# Install Keras version 2.3.1
%pip install keras==2.3.1

In [17]:
%pip install --upgrade pip

     ---------------------------------------- 2.1/2.1 MB 4.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2
Note: you may need to restart the kernel to use updated packages.


In [21]:


# Current stable release for CPU and GPU
%pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [24]:
%conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0
# Anything above 2.10 is not supported on the GPU on Windows Native
%python -m pip install "tensorflow=2.10"
# Verify install:
%python -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"



==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.7.4

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.4




Retrieving notices: ...working... done
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\chris\anaconda3

  added / updated specs:
    - cudatoolkit=11.2
    - cudnn=8.1.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    anaconda-anon-usage-0.4.0  |   py39h8ddfbff_0          26 KB
    anaconda-navigator-2.4.3   |   py39haa95532_0         5.5 MB
    conda-23.1.0               |   py39hcbf5309_0         912 KB  conda-forge
    cudatoolkit-11.2.2         |      h933977f_10       879.9 MB  conda-forge
    cudnn-8.1.0.77             |       h3e0f4f4_0       610.8 MB  conda-forge
    python_abi-3.9             |           2_cp39           4 KB  conda-forg



==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.7.3

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.3




Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\chris\anaconda3

  added / updated specs:
    - cudatoolkit=11.2
    - cudnn=8.1.0


The following NEW packages will be INSTALLED:

  anaconda-anon-usa~ pkgs/main/win-64::anaconda-anon-usage-0.4.0-py39h8ddfbff_0 
  cudatoolkit        conda-forge/win-64::cudatoolkit-11.2.2-h933977f_10 
  cudnn              conda-forge/win-64::cudnn-8.1.0.77-h3e0f4f4_0 
  python_abi         conda-forge/win-64::python_abi-3.9-2_cp39 

The following packages will be UPDATED:

  anaconda-navigator                   2.3.1-py39haa95532_0 --> 2.4.3-py39haa95532_0 

The following packages will be SUPERSEDED by a higher-priority channel:

  conda              pkgs/main::conda-23.1.0-py39haa95532_0 --> conda-forge::conda-23.1.



==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.7.3

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.3


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [23]:
import tensorflow
from tensorflow import keras


In [15]:
%tensorflow_version 2.x
import os
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential, Model 
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, MaxPooling1D, GlobalAveragePooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense, Input 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau 
from keras.callbacks import TensorBoard
from keras import backend as K 
from keras.optimizers import Adam, SGD, Adadelta
from keras.regularizers import l2, l1
import cv2
from keras.callbacks import CSVLogger
import sys
import time
import pickle
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from keras.utils.vis_utils import plot_model
import numpy as np
import matplotlib.pyplot as plt

from keras.applications.inception_v3 import InceptionV3

UsageError: Line magic function `%tensorflow_version` not found.


In [6]:
# Downloading zip file containing images
import zipfile
with zipfile.ZipFile('C:\\Users\\chris\\Onedrive\\Desktop\\ds340w\\images.zip', 'r') as zip_ref:
    zip_ref.extractall('C:\\Users\\chris\\Onedrive\\Desktop\\ds340w')

In [3]:
#Unzipping into "images" folder
!unzip -q images.zip -d images

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
# Loading images from Google Drive
#train_data_dir = '/content/images/train'
#validation_data_dir = '/content/images/test'
nb_train_samples = 67988 
nb_validation_samples = 22716
n_classes = 101
epochs = 10
batch_size = 75

In [16]:
# Checking image format: if RGB channel is coming first or last so, model will check first and then input shape will be feeded accordingly.
img_width = 299
img_height = 299

if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3) 

NameError: name 'K' is not defined

**Model**
<br>
- InceptionV3 base model, with imagenet weights loaded
- Average Pooling layer with pool size 8x8
- Dropout with probability 0.4
- Flatten layer
- Dense Layer of 101 neurons

In [ ]:
#Model that enable the freezing of the resnet layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(299, 299, 3)))
x = base_model.output
x = AveragePooling2D(pool_size=(8, 8))(x)
x = Dropout(.4)(x)
x = Flatten()(x)

predictions = Dense(n_classes,
                    kernel_regularizer=l2(0.005),
                    activity_regularizer=l1(0.005), 
                    activation='softmax')(x)

model = Model(input=base_model.input, output=predictions)

NameError: ignored

In [ ]:
#from tensorflow.keras.utils import plot_model
#plot_model(model, to_file='model_cnn.png')

In [ ]:
# Compile the model using Stochastic Gradiend Descent (SGD) optimizer
model.compile(
    optimizer=SGD(lr=.01, momentum=.9), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

In [ ]:
# Plot the model and save to file
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Train & Test Data Generators with image augmentation 

train_datagen = ImageDataGenerator(
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    zoom_range=[.8, 1],
    channel_shift_range=30,
    fill_mode='reflect')

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = 11,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = 11,
    class_mode='categorical')

Found 67988 images belonging to 101 classes.
Found 22716 images belonging to 101 classes.


In [ ]:
# Setup callbacks and logs 
checkpoint_path = "InceptionV3/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
cp = callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy',save_best_only=True,verbose=1, mode='max')
csv_logger = callbacks.CSVLogger('InceptionV3/InceptionV3.log')

In [ ]:
# Reduce LR if no improvement on the test accuracy is observed
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,
                              patience=2, min_lr=0.00001)

In [ ]:
# Loading best model weights
#model.load_weights('')

**Model Training**

In [ ]:
#Fitting
model.fit(train_generator,
          steps_per_epoch = nb_train_samples // batch_size,
          validation_data=validation_generator,
          validation_steps=nb_validation_samples // batch_size,
          epochs=10,
          verbose=1,
          callbacks=[cp_callback]
          )

Epoch 1/10
906/906 [==============================] - 3385s 4s/step - loss: 1.1828 - accuracy: 0.8552 - val_loss: 2.1801 - val_accuracy: 0.7048

Epoch 00001: val_accuracy improved from -inf to 0.70477, saving model to /content/drive/My Drive/Colab Notebooks/tesi_inceptionv3/weights-improvement-01-0.70.hdf5
Epoch 2/10
906/906 [==============================] - 3386s 4s/step - loss: 1.1012 - accuracy: 0.8752 - val_loss: 1.9721 - val_accuracy: 0.7109

Epoch 00002: val_accuracy improved from 0.70477 to 0.71088, saving model to /content/drive/My Drive/Colab Notebooks/tesi_inceptionv3/weights-improvement-02-0.71.hdf5
Epoch 3/10
906/906 [==============================] - 3379s 4s/step - loss: 1.0572 - accuracy: 0.8858 - val_loss: 2.0145 - val_accuracy: 0.7134

Epoch 00003: val_accuracy improved from 0.71088 to 0.71335, saving model to /content/drive/My Drive/Colab Notebooks/tesi_inceptionv3/weights-improvement-03-0.71.hdf5
Epoch 4/10
906/906 [==============================] - 3408s 4s/step - 

KeyboardInterrupt: ignored

In [ ]:
# Read log file
df = pd.read_csv('InceptionV3/InceptionV3.log')

In [ ]:
# Training and Test accuracy
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['epoch'], y=df['accuracy'],
                    mode='lines',
                    name='training'))

fig.add_trace(go.Scatter(x=df['epoch'], y=df['val_accuracy'],
                    mode='lines',
                    name='test'))


fig.update_layout(
    font_size = 20,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)

fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='Gray')
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='Gray')

In [ ]:
# Training and Test loss
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['epoch'], y=df['loss'],
                    mode='lines',
                    name='training loss'))

fig.add_trace(go.Scatter(x=df['epoch'], y=df['val_loss'],
                    mode='lines',
                    name='test loss'))


fig.update_layout(
    font_size = 20,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)

fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='Gray')
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='Gray')

**Model evaluation**

In [ ]:
#Evaluation
score = model.evaluate_generator(
    validation_generator, 
    nb_validation_samples/batch_size)

print(score)

[1.971716046333313, 0.716059148311615]
